Twitter competition using fastai

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *

copy files to working directory - this is to change working directory as well

In [2]:
#!cp  -a ../input/fast-ai-tweet/. ../working/

In [ ]:
!cp  -a ../input/twitter/. ../working/

In [ ]:
data_lm = TextDataBunch.from_csv('../working','DB2.csv', num_workers=0)

# save data bunch and load is as LMdatabunch

In [ ]:
data_lm.save()

In [ ]:
path='../working'

In [ ]:
data = TextLMDataBunch.load(path)
data.show_batch()

In [ ]:
data.vocab.itos[80:82]

And if we look at what a what's in our datasets, we'll see the tokenized text as a representation:

In [ ]:
data.train_ds[0][0]

In [ ]:
data.train_ds[0][0].data[:10]

In [ ]:
bs=48


In [ ]:
learn = language_model_learner(data, pretrained_model=URLs.WT103_1, drop_mult=0.3)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(5, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save('fit_head')

In [ ]:
learn.load('fit_head');

To complete the fine-tuning, we can then unfeeze and launch a new training.

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.save('fine_tuned')

How good is our model? Well let's try to see what it predicts after a few given words.

In [ ]:
learn.load('fit_head');

In [ ]:
TEXT = "fire around my house, help"
N_WORDS = 15
N_SENTENCES = 2

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

We have to save the model but also it's encoder, the part that's responsible for creating and updating the hidden state. For the next part, we don't care about the part that tries to guess the next word.

In [ ]:
learn.save_encoder('fine_tuned_enc')

## Classifier

Now, we'll create a new data object that only grabs the labelled data and keeps those labels. Again, this line takes a bit of time.

In [ ]:
!../working/tmp

In [ ]:
data_lm = TextDataBunch.from_csv('../working','train.csv', num_workers=0)
data_lm.save()

In [ ]:
data = TextClasDataBunch.load(path)
data.show_batch()

We can then create a model to classify those reviews and load the encoder we saved before.

In [ ]:
learn = text_classifier_learner(data, drop_mult=0.5)


In [ ]:
learn.load_encoder('fine_tuned_enc')


In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(20, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.save('first')

In [ ]:
learn.load('first');

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(6, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.save('second')

In [ ]:
learn.load('second');

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(25, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.save('third')

In [ ]:
learn.load('third');

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.predict("Just happened a terrible car crash")

In [ ]:
import pandas as pd
test = pd.read_csv("../working/test.csv")

In [ ]:
test['text'][0]

In [ ]:
bb = learn.predict(test['text'][0])

In [ ]:
res=[]

In [ ]:
for i in range(0,test.shape[0]):
    bb = learn.predict(test['text'][i])
    res.append(str(bb)[10])